In [1]:
# Concatenate ply data for each vh_object and create a label
# Category labels: axial components = 1, torso = 2, sheet = 3.

import os
from plyfile import PlyData
import h5py
import numpy as np


In [4]:
# data dir
ply_dir = r'/home/yiting/Documents/Data/Shapes/ply_1024/axial_component'
# save dir
h5_dir = r'/home/yiting/Documents/GitHub/dgcnn.pytorch/data/vhObject_ply_hdf5_1024'
file_name = 'ply_data_axial_component.h5'

In [5]:
# Write numpy array data and label to h5_filename
def save_h5(h5_filename, data, label, normal, data_dtype='uint8', label_dtype='uint8', normal_dtype='uint8'):
    h5_fout = h5py.File(h5_filename, "w")
    h5_fout.create_dataset(
            'data', data=data,
            compression='gzip', compression_opts=4,
            dtype=data_dtype)
    h5_fout.create_dataset(
            'label', data=label,
            compression='gzip', compression_opts=1,
            dtype=label_dtype)
    h5_fout.create_dataset(
            'normal', data=normal,
            compression='gzip', compression_opts=4,
            dtype=normal_dtype)
    h5_fout.close()

In [6]:
obj_names = os.listdir(ply_dir)
data_all = []
label_all = []
normal_all = []
for obj_name in obj_names:

    # Load PLY file
    ply_data = PlyData.read(os.path.join(ply_dir, obj_name))

    # Extract vertex data (assuming the PLY file has 'x', 'y', 'z' coordinates)
    vertex_data = ply_data['vertex'].data

    # Convert vertex coordinates to a NumPy array
    vertices = np.vstack([vertex_data['x'], vertex_data['y'], vertex_data['z']]).T

    # Check if the PLY file contains normal vectors ('nx', 'ny', 'nz')
    if 'nx' in vertex_data.dtype.names and 'ny' in vertex_data.dtype.names and 'nz' in vertex_data.dtype.names:
        normals = np.vstack([vertex_data['nx'], vertex_data['ny'], vertex_data['nz']]).T
    else:
        normals = None
        print("The PLY file does not contain normal vectors.")

    # Get label (axial_component = 1, torso = 2, sheet = 3)
    category = obj_name.split('_')[0]
    if category == 'axial':
        label = 1
    elif category == 'torso':
        label = 2
    elif category == 'sheet':
        label = 3
    else:
        print('Not in any category')

    # Append the data to lists
    data_all.append(vertices)
    label_all.append(label)
    normal_all.append(normals)

# Convert lists to NumPy arrays
data_all = np.array(data_all)  # Shape: (X, N, 3) where X is the number of ply objects and N is the number of vertices in each PLY
label_all = np.array(label_all) 

# If normal_all is not empty, concatenate into a NumPy array
normal_all = np.array(normal_all) if normal_all else None  # Shape: (X, N, 3) if normals are present

# Output the shapes of the arrays
print("Data shape:", data_all.shape)
print("Label shape:", label_all.shape)
print("Normal shape:", normal_all.shape if normal_all is not None else 'No normals')

h5_filename = os.path.join(h5_dir, file_name)
save_h5(h5_filename, data_all, label_all, normal_all)



Data shape: (145, 1024, 3)
Label shape: (145,)
Normal shape: (145, 1024, 3)


In [ ]:
# Read numpy array data and label from h5_filename
def load_h5_data_label_normal(h5_filename):
    f = h5py.File(h5_filename)
    data = f['data'][:]
    label = f['label'][:]
    normal = f['normal'][:]
    return (data, label, normal)

# Read numpy array data and label from h5_filename
def load_h5_data_label_seg(h5_filename):
    f = h5py.File(h5_filename)
    data = f['data'][:]
    label = f['label'][:]
    seg = f['pid'][:]
    return (data, label, seg)

# Read numpy array data and label from h5_filename
def load_h5(h5_filename):
    f = h5py.File(h5_filename)
    data = f['data'][:]
    label = f['label'][:]
    return (data, label)
